# Metaklasy

In [2]:
class Shape:
    def draw(self):
        pass

class Rectangle(Shape):
    _id = 'RECT'

    def __init__(self, width, height):
        self.width = width
        self.height = height        

    def draw(self):
        print(f'Drawing {Rectangle._id}({self.width}, {self.height})')

In [3]:
r = Rectangle(10, 20)

In [4]:
r.draw()

Drawing RECT(10, 20)


In [5]:
type(Rectangle)

type

In [6]:
Rectangle.__name__

'Rectangle'

To samo ale dynamicznie:

In [13]:
def rect_init(self, width, height):
        self.width = width
        self.height = height  

RectangleT = type('RectangleT', (Shape, ), {
        '_id': 'RECT',
        '__init__': rect_init,
        'draw' : lambda self: print(f'Drawing {RectangleT._id}({self.width}, {self.height})')
})

In [14]:
RectangleT.__name__

'RectangleT'

In [15]:
type(RectangleT)

type

In [16]:
r2 = RectangleT(10, 20)

In [17]:
r2.draw()

Drawing RECT(10, 20)


In [18]:
RectangleT._id

'RECT'

In [20]:
class AClass(metaclass=type):
    pass

In [ ]:
class Metaclass(type):
    def __new__(mcs, name, bases, class_dict):
        return super().__new__(mcs, name, bases, class_dict)

    @classmethod 
    def __prepare__(mcs, name, bases, **kwargs): 
        return super().__prepare__(name, bases, **kwargs)
    
    def __init__(cls, name, bases, class_dict, **kwargs): 
        super().__init__(name, bases, class_dict) 

    def __call__(cls, *args, **kwargs): 
        return super().__call__(*args, **kwargs)

In [ ]:
class RevealingMeta(type): 
    def __new__(mcs, name, bases, namespace, **kwargs): 
        print(mcs, "METACLASS __new__ called") 
        return super().__new__(mcs, name, bases, namespace) 
 
    @classmethod 
    def __prepare__(mcs, name, bases, **kwargs): 
        print(mcs, " METACLASS __prepare__ called") 
        return super().__prepare__(mcs, name, bases, **kwargs) 
 
    def __init__(cls, name, bases, namespace, **kwargs): 
        print(cls, " METACLASS __init__ called") 
        super().__init__(name, bases, namespace) 
 
    def __call__(cls, *args, **kwargs): 
        print(cls, " METACLASS __call__ called") 
        return super().__call__(*args, **kwargs) 

In [ ]:
class RevealingClass(metaclass=RevealingMeta):
    def __new__(cls):
        print(cls, "__new__ called")
        return super().__new__(cls)
    
    def __init__(self):
        print(self, "__init__ called")
        super().__init__()

<class '__main__.RevealingMeta'>  METACLASS __prepare__ called
<class '__main__.RevealingMeta'> METACLASS __new__ called
<class '__main__.RevealingClass'>  METACLASS __init__ called


In [ ]:
instance_rc = RevealingClass()

<class '__main__.RevealingClass'>  METACLASS __call__ called
<class '__main__.RevealingClass'> __new__ called
<__main__.RevealingClass object at 0x00000206CC90D850> __init__ called


In [36]:
class Meta(type):
    @property
    def some_property(cls):
        return f'property of {cls!r}'
    
    def some_method(self):
        return f'method of {self}'
    
class SomeClass(metaclass=Meta):
    pass

In [37]:
SomeClass.some_method()

"method of <class '__main__.SomeClass'>"

In [38]:
SomeClass.some_property

"property of <class '__main__.SomeClass'>"

In [39]:
SomeClass.__mro__

(__main__.SomeClass, object)

## Użycie metaklasy

In [30]:
from typing import Any, Mapping
import inflection

class CaseInterpolationDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        super().__setitem__(inflection.underscore(key), value)

class CaseInterpolatedMeta(type):
    @classmethod
    def __prepare__(metacls, __name: str, __bases: tuple[type, ...], **kwds: Any) -> Mapping[str, object]:
        return CaseInterpolationDict()
    

class MyUser(metaclass=CaseInterpolatedMeta):
    pass

class User(MyUser):    
    def __init__(self, firstName: str, lastName: str):
        self.firstName = firstName
        self.lastName = lastName

    def getDisplayName(self):
        return f"{self.firstName} {self.lastName}"
    
    def greetUser(self):
        return f"Hello {self.getDisplayName()}!"

In [33]:
User.__dict__

user = User("John", "Doe")

In [34]:
user.greet_user()

'Hello John Doe!'

# Metaklasy i kwargs

In [41]:
class AddClassAttributeMeta(type):
    def __new__(metaclass, name, bases, namespace, **kwargs):
        for k, v in kwargs.items():
            namespace.setdefault(k, v)
        return type.__new__(metaclass, name, bases, namespace)

    def __init__(metaclass, name, bases, namespace, **kwargs):
        type.__init__(metaclass, name, bases, namespace)

In [42]:
class WithArgs(metaclass=AddClassAttributeMeta, a = 665):
    pass

In [43]:
WithArgs.a

665

## `__init_subclass__`

In [44]:
class AddClassAttribute:
    def __init_subclass__(cls, **kwargs) -> None:
        super().__init_subclass__()

        for k, v in kwargs.items():
            setattr(cls, k, v)


class WithAttributes(AddClassAttribute, a = 667):
    pass

In [45]:
WithAttributes.a

667

# namedtuple - how it works

In [46]:
from typing import Iterable, Iterator, Union


FieldNames = Union[str, Iterable[str]]


def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()

    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')

    return tuple(names)

def record_factory(cls_name, field_names: FieldNames):
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)

        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join(f"{name}={value!r}" for name,
                           value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f"{cls_name}({values})"

    cls_attrs = dict(
        __slots__ = slots,
        __init__ = __init__,
        __repr__ = __repr__,
        __iter__ = __iter__
    )

    return type(cls_name, (object, ), cls_attrs)

In [47]:
Point = record_factory('Point', 'x y')

In [48]:
pt1 = Point(20, 30)

In [49]:
pt1

Point(x=20, y=30)

In [50]:
for i in pt1:
    print(i)

20
30


In [51]:
Point.__dict__

mappingproxy({'__slots__': ('x', 'y'),
              '__init__': <function __main__.record_factory.<locals>.__init__(self, *args, **kwargs) -> None>,
              '__repr__': <function __main__.record_factory.<locals>.__repr__(self)>,
              '__iter__': <function __main__.record_factory.<locals>.__iter__(self) -> Iterator[Any]>,
              '__module__': '__main__',
              'x': <member 'x' of 'Point' objects>,
              'y': <member 'y' of 'Point' objects>,
              '__doc__': None})

## NamedTuple

In [57]:
class MetaBunch(type):
    def __new__(mcl, classname, bases, classdict):
        
        # Define as local functions __init__ & __repr__ that we'll use 
        # in the new class
        def __init__(self, **kwargs):
            """__init__ is simple: first, set attributes without
               explicit values to their defaults; then, set those
               explicitly passed in kw.
            """
            for k in self.__dflts__:
                if not k in kwargs:
                    setattr(self, k, self.__dflts__[k])
            for k in kwargs:
                setattr(self, k, kwargs[k])

        def __repr__(self):
            """__repr__ is minimal: shows only attributes that
               differ from default values, for compactness.
            """
            rep = [f'{k}={getattr(self, k)!r}' for k in self.__dflts__ if getattr(self, k) != self.__dflts__[k]]
            return f"{classname}({', '.join(rep)})"
        
        newdict = { 
            '__slots__': [], 
            '__dflts__': {},
            '__init__' : __init__, 
            '__repr__' : __repr__,
        }
        for k in classdict:
            if k.startswith('__') and k.endswith('__'):
                if k in newdict:
                    warnings.warn(f"Cannot set attr {k!r} in bunch-class {classname!r}")
                else:
                    newdict[k] = classdict[k]
            else:
                newdict['__slots__'].append(k)
                newdict['__dflts__'][k] = classdict[k]

        return super().__new__(mcl, classname, bases, newdict)

class Bunch(metaclass=MetaBunch):
    pass

In [58]:
class Point(Bunch):
    x = 0
    y = 0
    color = 'red'

pt = Point(x=10, y=20)

In [59]:
pt.x

10

In [60]:
pt.y

20

In [61]:
pt.color

'red'

In [62]:
pt

Point(x=10, y=20)

# ABC

In [66]:
def abstractmethod(function):
    function.__abstract__ = True
    return function 

In [69]:
import functools


class AbstractMeta(type):
    def __new__(metaclass, name, bases, namespace):
        # Create the class instance
        print(f"__new__({metaclass}, {name}, {bases!r}, {namespace!r})")
        cls = super().__new__(metaclass, name, bases, namespace)

        # Collect all local method marked as abstract
        abstracts = set()
        for k, v in namespace.items():
            if getattr(v, '__abstract__', False):
                abstracts.add(k)

        # Look for abstract methods in the base classes and add them to the list of abstracts
        for base in bases:
            for k in getattr(base, '__abstracts__', ()):
                v = getattr(cls, k, None)
                if getattr(v, '__abstract__', False):
                    abstracts.add(k)

        # Store the abstracts in a frozenset so the cannot be modified
        cls.__abstracts__ = frozenset(abstracts)

        # Decorate the __new__ function to check if all abstract functions were implemented
        original_new = cls.__new__
        @functools.wraps(original_new)
        def new(self, *args, **kwargs):
            for k in self.__abstracts__:
                v = getattr(self, k)
                if getattr(v, '__abstract__', False):
                    raise RuntimeError(f'{k} is not implemented')
            return original_new(self, *args, **kwargs)
        
        cls.__new__ = new
        return cls

In [74]:
class Shape(metaclass=AbstractMeta):

    @abstractmethod
    def draw(self) -> None:
        pass


class RotableShape(Shape):
    @abstractmethod
    def rotate(self, angle):
        pass

    def draw(self):
        print("Drawing shape")

class Rectangle(RotableShape):
    def rotate(self, angle):
        print("Rotate rectangle")

r = Rectangle()

__new__(<class '__main__.AbstractMeta'>, Shape, (), {'__module__': '__main__', '__qualname__': 'Shape', 'draw': <function Shape.draw at 0x00000206CCFAED40>})
__new__(<class '__main__.AbstractMeta'>, RotableShape, (<class '__main__.Shape'>,), {'__module__': '__main__', '__qualname__': 'RotableShape', 'rotate': <function RotableShape.rotate at 0x00000206CCFAD120>, 'draw': <function RotableShape.draw at 0x00000206CCFAE700>})
__new__(<class '__main__.AbstractMeta'>, Rectangle, (<class '__main__.RotableShape'>,), {'__module__': '__main__', '__qualname__': 'Rectangle', 'rotate': <function Rectangle.rotate at 0x00000206CCFAEB60>})


In [75]:
r.draw()

Drawing shape


In [77]:
r.rotate(45)

Rotate rectangle
